In [4]:
import sys
sys.path.append('/home/alexj/hgcal-dev')

In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import torch

In [1]:
import wandb

In [2]:
api = wandb.Api()

In [3]:
run = api.run('uw-hgcal/hgcal-spvcnn/2tp2l5gx')

In [10]:
run.file('hgcal-spvcnn/2tp2l5gx/checkpoints/epoch=14.ckpt').download()

<_io.TextIOWrapper name='./hgcal-spvcnn/2tp2l5gx/checkpoints/epoch=14.ckpt' mode='r' encoding='UTF-8'>

In [11]:
run.files()

<Files uw-hgcal/hgcal-spvcnn/2tp2l5gx (9)>

In [2]:
from hgcal_dev.utils.experiment import Experiment

In [3]:
run_path = Path('/home/alexj/outputs/wandb/run-20210119_182022-2bhsqvop')
ckpt_path = Path('/home/alexj/outputs/wandb/latest-run/files/hgcal-spvcnn/2bhsqvop/checkpoints/epoch=14.ckpt')
experiment = Experiment(run_path, ckpt_path)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: alexschuy (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
model = experiment.model
datamodule = experiment.datamodule
model.cuda(0)
model.eval()
datamodule.batch_size = 1
datamodule.prepare_data()
datamodule.setup(stage='fit')

In [6]:
dataloader = datamodule.val_dataloader()
dataset = dataloader.dataset


In [7]:
def get_event(i):
    dataloader = datamodule.val_dataloader()
    dataset = dataloader.dataset
    raw_event = np.load(dataset.events[i])
    return dataset[i], raw_event

In [91]:
batch, raw = get_event(0)

In [92]:
features = batch['features']
features.C = torch.tensor(features.C)
features.F = torch.tensor(features.F)

In [93]:

features = batch['features'].to(model.device)
class_prediction, embed_prediction = model(features)

In [94]:
class_prediction

tensor([[-0.6026, -0.2733,  0.2126, -1.6268],
        [ 1.9507, -0.7544, -2.1751, -4.2910],
        [-0.3824,  0.2650, -0.8349, -0.7370],
        ...,
        [ 2.5427, -1.2317, -2.3116, -5.4669],
        [ 2.7060, -1.3893, -2.4958, -5.9926],
        [ 1.9360, -0.7593, -2.1121, -3.6693]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [95]:
class_prediction = torch.argmax(class_prediction, dim=1)

In [96]:
class_prediction

tensor([2, 0, 1,  ..., 0, 0, 0], device='cuda:0')

In [97]:
torch.unique(class_prediction, return_counts=True)

(tensor([0, 1, 2], device='cuda:0'),
 tensor([  252, 19392,   655], device='cuda:0'))

In [98]:
raw_x, raw_labels = raw['x'], raw['y']

In [101]:
full_class_prediction = class_prediction.cpu().numpy()[batch['inverse_map'].F]

In [103]:
(raw_labels[:, 0] == full_class_prediction).sum() / full_class_prediction.shape[0]

0.03347179289026275

In [14]:
dataloader = datamodule.val_dataloader()
class_predictions = []
embed_predictions = []
targets = []
with torch.no_grad():
        for i, (batch, event_path) in tqdm(enumerate(zip(dataloader, dataloader.dataset.events))):
            features = batch['features'].to(model.device)
            labels = batch['labels'].to(model.device)
            targets.append(labels)
            class_prediction, embed_prediction = model(features)
            class_prediction = torch.argmax(class_prediction, dim=1)
            class_predictions.append(class_prediction.cpu())
            embed_predictions.append(embed_prediction.cpu())

985it [01:17, 12.71it/s]


In [ ]:
targets = []
with torch.no_grad():
        for i, (batch, event_path) in tqdm(enumerate(zip(dataloader, dataloader.dataset.events))):
            features = batch['features'].to(model.device)
            labels = batch['labels'].to(model.device)
            class_prediction, embed_prediction = model(features)
            class_predictions.append(class_prediction.cpu())
            embed_predictions.append(embed_prediction.cpu())

In [8]:
from hgcal_dev.clustering.mean_shift_cosine_gpu import MeanShiftCosine

In [9]:
clusterer = MeanShiftCosine()


In [11]:
instance_predictions = []
for embed in tqdm(embed_predictions):
    clusterer.fit(embed)
    instance_predictions.append(clusterer.labels_)

100%|██████████| 985/985 [02:14<00:00,  7.34it/s]


In [12]:
from hgcal_dev.metrics.instance import PanopticQuality

In [13]:
metric = PanopticQuality(num_classes=2, ignore_index=-1, min_points=1)

In [15]:
import torch

In [17]:
instance_predictions[0]

array([0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0,
       0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2,
       0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0,
       0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 2, 0, 2, 2,
       2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 0, 2, 0, 0,
       0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 1, 0, 0, 1, 0, 0,
       0, 4, 0, 4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 4, 0, 4, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2,
       2, 0, 1, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 1, 1, 0, 4, 4, 2,
       0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 4, 4, 0, 2, 2, 0, 0, 0,

In [22]:
targets[0].F[:, 1].unique()

tensor([-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40.,
        41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
        55., 56., 57., 58., 59., 60., 61.], device='cuda:0')

In [ ]:
outputs = (class_predictions[0], instance_predictions[0])
targets = ()